In [0]:
# %pip install -U -qqqq databricks-langchain langgraph databricks-agents mlflow-skinny[databricks]
# dbutils.library.restartPython()

### Using langraph create_react_agent

In [0]:
import json
from typing import Any, Generator
from uuid import uuid4

import mlflow
from databricks_langchain import ChatDatabricks, UCFunctionToolkit
from langchain_core.messages import AIMessageChunk
from langgraph.prebuilt import create_react_agent

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

############################################
# Define LLM and tools
############################################
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# UC_TOOL_NAMES = ["system.ai.python_exec"]
# uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
# tools = uc_toolkit.tools


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"The weather in {city} is sunny."

def get_time(zone: str) -> str:
    """Get current time for a given timezone."""
    return f"The current time in {zone} is 2:00 PM."


    
  

tools = [get_weather,get_time]

system_prompt = "You are a helpful assistant that can run Python code."

############################################
# Build the ReAct agent automatically
############################################
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt,  # this injects your system prompt
)





In [0]:
############################################
# Register the agent with MLflow
############################################
from helpers import LangGraphResponsesAgent
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(agent_graph)
mlflow.models.set_model(AGENT)

In [0]:
result = AGENT.predict({"input": [{"role": "user", "content": "What’s the weather in Paris and the time in EST? and search wikipedia for fun facts about paris"}]})
print(result.model_dump(exclude_none=True))

https://python.langchain.com/docs/integrations/tools/

In [0]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


# --- create Wikipedia tool ---
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [0]:
tools = [get_weather,get_time,wiki]

prompt = """
You are a helpful AI assistant. You can think step-by-step and use the following tools to answer user questions:

1. get_weather(city: str) — returns the current weather in the given city.
2. get_time(zone: str) — returns the current time for the given timezone.
3. WikipediaQueryRun(input: str) — searches Wikipedia and returns relevant information.

Rules:
- If a question needs factual information or background knowledge, prefer WikipediaQueryRun.
- If the user asks about weather, use get_weather.
- If the user asks about time, use get_time.
- Always return a clear final answer to the user after using tools.
- Think carefully about which tool is best for each part of the question. 
- You can use multiple tools in one conversation if the question needs it.
- Do not guess values you can fetch with a tool — always call the correct tool.

Follow the ReAct pattern:
- First, think about what is needed.
- Then, call the right tool with correct arguments.
- Finally, summarize the result back to the user.
"""


############################################
# Build the ReAct agent automatically
############################################
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=prompt,  # this injects your system prompt
)


In [0]:
############################################
# Register the agent with MLflow
############################################
from helpers import LangGraphResponsesAgent
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(agent_graph)
mlflow.models.set_model(AGENT)

In [0]:
result = AGENT.predict({"input": [{"role": "user", "content": "What’s the weather in Paris and the time in EST? and search wikipedia for fun facts about paris"}]})
print(result.model_dump(exclude_none=True))